# `SENTIMENTAL ANALYSIS`

#### `IMPORTIN LIBRARIES`

In [279]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import pickle
import string
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### `IMPORTING DATASET`

In [280]:
dataset=pd.read_csv("train.csv")

In [281]:
dataset.head(20)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...


### `PRE-PROCESSING`

##### `PRE PROCESSING VARIABLE SETTING`

In [282]:
STOP_WORDS=stopwords.words('english')
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [283]:
dataset['tweet'] = dataset['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [284]:
# removing stop words
dataset.tweet=dataset.tweet.apply(lambda x : " ".join (x for x in x.split() if x not in STOP_WORDS))
dataset

,id,label,tweet
0,1,0,user father dysfunctional selfish drags kids d...
1,2,0,user user thanks lyft credit cant use cause do...
2,3,0,bihday majesty
3,4,0,model love u take u time ur
4,5,0,factsguide society motivation
...,...,...,...
31957,31958,0,ate user isz youuu
31958,31959,0,see nina turner airwaves trying wrap mantle ge...
31959,31960,0,listening sad songs monday morning otw work sad
31960,31961,1,user sikh temple vandalised calgary wso condem...


In [287]:
# Stemming
st=PorterStemmer()
dataset['tweet'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0    user father dysfunct selfish drag kid dysfunct...
1    user user thank lyft credit cant use caus dont...
2                                       bihday majesti
3                          model love u take u time ur
4                              factsguid societi motiv
Name: tweet, dtype: object

In [288]:
# Lemmatizing

dataset['tweet']=dataset['tweet'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
dataset['tweet']

0        user father dysfunctional selfish drag kid dys...
1        user user thanks lyft credit cant use cause do...
2                                           bihday majesty
3                              model love u take u time ur
4                            factsguide society motivation
                               ...                        
31957                                   ate user isz youuu
31958    see nina turner airwave trying wrap mantle gen...
31959       listening sad song monday morning otw work sad
31960    user sikh temple vandalised calgary wso condem...
31961                                    thank user follow
Name: tweet, Length: 31962, dtype: object

In [289]:
dataset=dataset.drop(["id"],axis=1)
dataset

,label,tweet
0,0,user father dysfunctional selfish drag kid dys...
1,0,user user thanks lyft credit cant use cause do...
2,0,bihday majesty
3,0,model love u take u time ur
4,0,factsguide society motivation
...,...,...
31957,0,ate user isz youuu
31958,0,see nina turner airwave trying wrap mantle gen...
31959,0,listening sad song monday morning otw work sad
31960,1,user sikh temple vandalised calgary wso condem...


In [290]:
col_list=list(dataset.columns)
dataset["label"],dataset["tweet"]=dataset["tweet"],dataset["label"]
dataset.columns=col_list
dataset

,label,tweet
0,user father dysfunctional selfish drag kid dys...,0
1,user user thanks lyft credit cant use cause do...,0
2,bihday majesty,0
3,model love u take u time ur,0
4,factsguide society motivation,0
...,...,...
31957,ate user isz youuu,0
31958,see nina turner airwave trying wrap mantle gen...,0
31959,listening sad song monday morning otw work sad,0
31960,user sikh temple vandalised calgary wso condem...,1


### `SPLITTING DATA FOR TRAINING AND TESTING`

#### `LSTM MODEL`

In [294]:
X=dataset.tweet
Y=dataset.label

In [295]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=0)

In [296]:
def tokenizing_padding(x):
    max_fatures = 2000
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(dataset['text'].values)
    X = tokenizer.texts_to_sequences(dataset['text'].values)
    X = pad_sequences(X)
    return X,tokenizer


In [297]:
vocabulary_size = 5000
count_vector = CountVectorizer(max_features=vocabulary_size,
                                preprocessor=lambda x: x,
                               tokenizer=lambda x: x) 
X_train = count_vector.fit_transform(X_train).toarray()
X_test = count_vector.transform(X_test).toarray()

TypeError: 'int' object is not iterable

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=0)